# PyData London 2017

# Foreign Function Interface Generator: Generating Python bindings from C++

### Jonathan B Coe
### jbcoe@ffig.org

## https://github.com/ffig/ffig

We want to run C++ code in Python without doing any extra work.

## Gathering Input

Write a C++ class out to a file in the current working directory

In [ ]:
outputfile = "Shape.h"

In [ ]:
%%file $outputfile
#include <stdexcept>
#include <string>

struct Shape
{
  virtual ~Shape() = default;
  virtual double area() const = 0;
  virtual double perimeter() const = 0;
  virtual const char* name() const = 0;
} __attribute__((annotate("GENERATE_C_API")));

static const double pi = 4.0;

class Circle : public Shape
{
  const double radius_;

public:
  double area() const override
  {
    return pi * radius_ * radius_;
  }

  double perimeter() const override
  {
    return 2 * pi * radius_;
  }

  const char* name() const override
  {
    return "Circle";
  }

  Circle(double radius) : radius_(radius)
  {
    if ( radius < 0 ) 
    { 
      std::string s = "Circle radius \"" + std::to_string(radius_) + "\" must be non-negative.";
      throw std::runtime_error(s);
    }
  }
};

Compile our header to check it's valid C++

In [ ]:
%%sh
clang++ -x c++ -fsyntax-only -std=c++14 Shape.h 

Read the code using libclang

In [ ]:
import sys
sys.path.insert(0,'../ffig')

In [ ]:
import clang.cindex

index = clang.cindex.Index.create()
translation_unit = index.parse(outputfile, ['-x', 'c++', '-std=c++14'])

In [ ]:
import asciitree

def node_children(node):
    return (c for c in node.get_children() if c.location.file.name == outputfile)

print asciitree.draw_tree(translation_unit.cursor,
  lambda n: [c for c in node_children(n)],
  lambda n: "%s (%s)" % (n.spelling or n.displayname, str(n.kind).split(".")[1]))


Turn the AST into some easy to manipulate Python classes

In [ ]:
import cppmodel

In [ ]:
model = cppmodel.Model(translation_unit)

In [ ]:
[f.name for f in model.functions][-5:]

In [ ]:
[c.name for c in model.classes][-5:]

In [ ]:
shape_class = [c for c in model.classes if c.name=='Shape'][0]

In [ ]:
["{}::{}".format(shape_class.name,m.name) for m in shape_class.methods]

## Code Generation

We now have some input to use in a code generator.

Look at the templates the generator uses

In [ ]:
%cat ../ffig/templates/json.tmpl

Run the code generator

In [ ]:
%%sh
python ../ffig/FFIG.py -b _c.h.tmpl _c.cpp.tmpl json.tmpl python -m Shape -i Shape.h

See what it created

In [ ]:
%ls

In [ ]:
%cat Shape.json

Build some bindings with the generated code.

In [ ]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
add_library(Shape_c SHARED Shape_c.cpp)

In [ ]:
%%sh
cmake . 
cmake --build .

In [ ]:
%%sh
nm -U libShape_c.dylib | c++filt

In [ ]:
cat Shape/_py3.py

In [ ]:
%%python2
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

In [ ]:
%%python3
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

In [ ]:
%%script pypy
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

In [ ]:
%%script /opt/intel/intelpython35/bin/python
import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

In [ ]:
from Shape import *

In [ ]:
c = Circle(-8)

## FFIG needs you!

FFIG is MIT-licensed and hosted on GitHub.

Contributions, issues and feedback are very welcome.